# Import modules

In [3]:
import numpy as np
import MDAnalysis as mda
import os

# Preparations

## read in box .pdb to modify

In [1]:
input_path = str(input('enter path to pdb file \n')) # snapshot.pdb

enter path to pdb file 
 snapshot.pdb


In [2]:
readpdb = []
filepath = open(input_path, 'r') #read in .pdb file as 'topology'
for line in filepath.readlines():
    line = line.strip()
    readpdb.append(line)
filepath.close()

In [6]:
coords_start = [line_no for line_no,line in enumerate(readpdb) if 'ATOM' in line]
header_loc = coords_start[0]
print(header_loc)

2


In [7]:
footer = len(readpdb) - 1 - coords_start[-1]
print(footer)

1


In [8]:
atom_data = np.genfromtxt(str(input_path) ,dtype=(str),usecols=(0),skip_header=header_loc,skip_footer=footer) #ATOM
atom_no = np.genfromtxt(str(input_path) ,dtype=(int),usecols=(1),skip_header=header_loc,skip_footer=footer) #Atom no.
atom_type = np.genfromtxt(str(input_path) ,dtype=(str),usecols=(2),skip_header=header_loc,skip_footer=footer) #Atom type
residue_type = np.genfromtxt(str(input_path) ,delimiter=(4,1,6,1,4,1,4,1,4,12,8,8,1,23), dtype=str, usecols=6, skip_header=header_loc,skip_footer=footer) #residue type
residue_no = np.genfromtxt(str(input_path) ,dtype=int,usecols=(4),skip_header=header_loc,skip_footer=footer) #resid no.
x = np.genfromtxt(str(input_path) ,dtype=(float),usecols=(5),skip_header=header_loc,skip_footer=footer) #x
y = np.genfromtxt(str(input_path) ,dtype=(float),usecols=(6),skip_header=header_loc,skip_footer=footer) #y
z = np.genfromtxt(str(input_path) ,dtype=(float),usecols=(7),skip_header=header_loc,skip_footer=footer) #z
comments = np.genfromtxt(str(input_path), delimiter=(4,1,6,1,8,6,12,8,8,1,20),dtype=(str),usecols=(10),skip_header=header_loc,skip_footer=footer) #rest

In [9]:
atom_no #check format

array([   1,    2,    3, ..., 6023, 6024, 6025])

In [11]:
residue_type

array(['MP_2', 'MP_2', 'MP_2', ..., 'TIP3', 'TIP3', 'TIP3'], dtype='<U4')

## separate water and protein

In [13]:
water_resname = 'TIP3'

In [14]:
water = np.char.find(residue_type, water_resname, start=0, end=None) #find all entries w/ SOL

In [15]:
sol_loc = np.argwhere(water > -1) #get indices for SOL

In [16]:
atom_type[sol_loc[0]] #first SOL

array(['OH2'], dtype='<U3')

In [17]:
start = sol_loc[0]

In [18]:
int(start) # needs int formatting for slicing

25

## fix residue numbers

In [19]:
res_no = np.add(residue_no[int(start):],np.full(len(residue_no[int(start):]),3,dtype=int))

In [20]:
res_no

array([   4,    4,    4, ..., 2025, 2025, 2025])

In [21]:
res_no_2 = residue_no[:int(start)]

In [22]:
residue_no = np.concatenate((res_no_2, res_no))

## solute dimensions

In [28]:
solute_coords = np.stack((x[:int(start)], y[:int(start)], z[:int(start)]),axis=-1) # merge xyz of protein

In [29]:
solute_coords # xyz of protein

array([[ 7.853, 23.148, 28.251],
       [ 8.068, 21.493, 28.357],
       [ 9.008, 23.938, 28.829],
       [ 6.497, 23.401, 28.933],
       [ 7.697, 23.372, 26.755],
       [ 8.449, 21.053, 29.656],
       [ 7.96 , 20.101, 29.953],
       [ 8.182, 21.76 , 30.47 ],
       [ 9.543, 20.901, 29.775],
       [ 8.66 , 21.344, 24.09 ],
       [ 9.191, 21.601, 25.454],
       [ 7.668, 20.846, 24.135],
       [ 9.321, 20.615, 23.575],
       [ 8.402, 22.313, 23.611],
       [10.169, 21.942, 25.355],
       [ 9.203, 20.709, 25.989],
       [ 8.645, 22.334, 25.949],
       [ 5.32 , 21.138, 31.287],
       [ 5.343, 21.296, 29.823],
       [ 4.453, 21.737, 31.638],
       [ 6.271, 21.558, 31.679],
       [ 5.163, 20.064, 31.527],
       [ 5.696, 22.232, 29.538],
       [ 4.366, 21.313, 29.467],
       [ 5.856, 20.535, 29.334]])

In [30]:
solute_types = residue_type[:len(solute_coords)]

In [31]:
solute_types

array(['MP_2', 'MP_2', 'MP_2', 'MP_2', 'MP_2', 'MP_2', 'MP_2', 'MP_2',
       'MP_2', 'MAM1', 'MAM1', 'MAM1', 'MAM1', 'MAM1', 'MAM1', 'MAM1',
       'MAM1', 'MAM2', 'MAM2', 'MAM2', 'MAM2', 'MAM2', 'MAM2', 'MAM2',
       'MAM2'], dtype='<U4')

In [34]:
solute_types[:9]

array(['MP_2', 'MP_2', 'MP_2', 'MP_2', 'MP_2', 'MP_2', 'MP_2', 'MP_2',
       'MP_2'], dtype='<U4')

In [40]:
solute_types[9:17]

array(['MAM1', 'MAM1', 'MAM1', 'MAM1', 'MAM1', 'MAM1', 'MAM1', 'MAM1'],
      dtype='<U4')

In [41]:
solute_types[17:25]

array(['MAM2', 'MAM2', 'MAM2', 'MAM2', 'MAM2', 'MAM2', 'MAM2', 'MAM2'],
      dtype='<U4')

## fix residue numbers

In [45]:
residue_no[:9] = 1

In [46]:
residue_no[9:17] = 2

In [47]:
residue_no[17:25] = 3

In [49]:
residue_no[:30] # fixed

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
       3, 3, 3, 4, 4, 4, 5, 5])

In [50]:
solute_coords[:9]

array([[ 7.853, 23.148, 28.251],
       [ 8.068, 21.493, 28.357],
       [ 9.008, 23.938, 28.829],
       [ 6.497, 23.401, 28.933],
       [ 7.697, 23.372, 26.755],
       [ 8.449, 21.053, 29.656],
       [ 7.96 , 20.101, 29.953],
       [ 8.182, 21.76 , 30.47 ],
       [ 9.543, 20.901, 29.775]])

In [51]:
solute1_coords = solute_coords[:9] # only mp2 w/o mamm

In [52]:
xavg = np.mean(solute1_coords[:,0])
yavg = np.mean(solute1_coords[:,1])
zavg = np.mean(solute1_coords[:,2])

In [53]:
avg = []
avg.append(xavg)
avg.append(yavg)
avg.append(zavg)

In [54]:
avg = np.array(avg)

In [55]:
avg

array([ 8.13966667, 22.12966667, 28.99766667])

### add 8 Angström in each direction

In [56]:
minus = np.full((len(avg)),-8.,dtype=float) # arrays for max box dims (+/- 8 angström)
plus = np.full((len(avg)),8,dtype=float)

In [57]:
minus

array([-8., -8., -8.])

In [58]:
avg_minus = np.add(avg,minus) # coords of box
avg_plus = np.add(avg,plus)

In [59]:
avg_minus

array([ 0.13966667, 14.12966667, 20.99766667])

In [60]:
avg_plus

array([16.13966667, 30.12966667, 36.99766667])

## prep box for atom selection

In [61]:
boxcoords = np.stack((x[int(start):], y[int(start):], z[int(start):]),axis=-1) # merge xyz of box

In [62]:
boxcoords.shape # xyz of simulation box atoms

(6000, 3)

In [63]:
boxcoords[:,0].shape

(6000,)

# locate atoms outside box

In [64]:
def find_atoms_outside_box(boxcoords):
    """
    finds coordinates which are outside of the defined box grabs their indices
    
    Parameters
    ----------
    boxcoords : np.array
        holds the system's atomic coordinates
        
    Returns
    -------
    combined_indices : np.array
        indices of all coords which are outsied of the defined box
    """
    # indices of atoms outside box (x,y,z)
    x_below = [] #coord-8
    y_below = []
    z_below = []

    x_above = [] #coord+8
    y_above = []
    z_above = []
    
    i = 0
    while i < 3: # x,y,z columns
        for index, coord in enumerate(boxcoords[:,i]):

            # grab indices of atoms outside the box
            if coord < avg_minus[i]: 
                if i == 0:
                    x_below = np.append(x_below, index)
                if i == 1:
                    y_below = np.append(y_below, index)
                if i == 2:
                    z_below = np.append(z_below, index)
        i = i + 1

    i = 0
    while i < 3:
        for index, coord in enumerate(boxcoords[:,i]):

            if coord > avg_plus[i]:
                if i == 0:
                    x_above = np.append(x_above,index)
                if i == 1:
                    y_above = np.append(y_above,index)
                if i == 2:
                    z_above = np.append(z_above,index)
        i = i + 1
        
    combined_indices = np.unique(np.sort(np.concatenate((x_below, y_below, z_below, x_above, y_above, z_above)))) #combine all findings into 1 array, sort ascending, remove duplicates
    combined_indices = combined_indices.astype('int') #format as integer for slicing
    
    return combined_indices

In [65]:
combined_indices = find_atoms_outside_box(boxcoords)

In [66]:
len(combined_indices) # no of coords outside box

5603

In [67]:
boxcoords[combined_indices] #coords of atoms outside box

array([[21.684, 38.954,  2.92 ],
       [21.76 , 39.462,  3.727],
       [22.561, 38.981,  2.536],
       ...,
       [29.849, 11.759, 13.495],
       [30.61 , 12.153, 13.921],
       [29.258, 11.538, 14.215]])

In [68]:
solvent_data = np.rec.fromarrays((atom_data[int(start):], atom_no[int(start):], atom_type[int(start):], residue_type[int(start):], residue_no[int(start):], x[int(start):], y[int(start):], z[int(start):], comments[int(start):])) # box w/o prot

In [69]:
solvent_outside_box = solvent_data[combined_indices]

In [70]:
solvent_outside_box # atoms outside box

rec.array([('ATOM',   26, 'OH2', 'TIP3',    4, 21.684, 38.954,  2.92 , ' 1.00  0.00      WAT'),
           ('ATOM',   27, 'H1', 'TIP3',    4, 21.76 , 39.462,  3.727, ' 1.00  0.00      WAT'),
           ('ATOM',   28, 'H2', 'TIP3',    4, 22.561, 38.981,  2.536, ' 1.00  0.00      WAT'),
           ...,
           ('ATOM', 6023, 'OH2', 'TIP3', 2025, 29.849, 11.759, 13.495, ' 1.00  0.00      WAT'),
           ('ATOM', 6024, 'H1', 'TIP3', 2025, 30.61 , 12.153, 13.921, ' 1.00  0.00      WAT'),
           ('ATOM', 6025, 'H2', 'TIP3', 2025, 29.258, 11.538, 14.215, ' 1.00  0.00      WAT')],
          dtype=[('f0', '<U4'), ('f1', '<i8'), ('f2', '<U3'), ('f3', '<U4'), ('f4', '<i8'), ('f5', '<f8'), ('f6', '<f8'), ('f7', '<f8'), ('f8', '<U20')])

In [71]:
def delete_atoms_outside_box(sliced_pdb_data, water_resname, cation_resname, anion_resname):
    """
    Screens data read-in from a pdb file which has been sliced by indices of atoms outside the defined box. Only works for 3-point water model and 1-atomic ions. 
    Lipid screening is not really implemented (would require number of atoms of each lipid) 
    
    Parameters
    ----------
    sliced_pdb_data : np.recarray
        read-in data from pdb which has been sliced by indices of atoms outside the defined box
    water_resname : str
        residue name of solvent(water) molecules, e.g. 'TIP3'
    cation_resname : str
        residue name of 1-atomic cation, e.g. 'NA'
    anion_resname : str
        residue name of 1-atomic anion, e.g. 'CL'
        
    Returns
    -------
    molecules2del : list of lists
        contains water2del, ions2del, lipids2del
    molecules : list of lists
        cations, anions, lipids
    """
    assert water_resname not in ['TIP4', 'TIP5'], 'only 3-point water model supported'
    assert cation_resname in ['NA', 'SOD', 'K', 'POT'], 'cation {} not supported'.format(cation_resname)
    assert anion_resname in ['CL', 'BR'], 'anion {} not supported'.format(anion_resname)
    
    water2del = [] # where the residue numbers of waters to delete are stored
    ions2del = [] # where the residue numbers of ions to delete are stored
    lipids2del = [] # where the atom numbers of ovelapping lipid atoms are stored
    cations = [] # where the cations go
    anions = [] # where the anions go
    lipids = [] # where the lipids go

    i = 0
    while i < len(solvent_outside_box):

        if (water_resname in str(solvent_outside_box[i][3])) == True: # entry is water atom

            water2del = np.append(water2del, solvent_outside_box[i][4]) # append residue number
            
            try:

                if solvent_outside_box[i+1][4] == solvent_outside_box[i][4]: # following entry belongs to same water molecule
                    
                    if solvent_outside_box[i+2][4] == solvent_outside_box[i][4]: # entire water molecule is in overlap list

                        i = i + 3 # skip next 2 atoms
                        
                    else:
                        i = i + 2 # skip next atom
                    
                else: # only one atom of water molecule is in overlap list
                    i = i + 1 # move to next atom

            except IndexError:

                continue

        elif (cation_resname in str(solvent_outside_box[i][3]) or anion_resname in str(solvent_outside_box[i][3])) == True: # entry is ion

            ions2del = np.append(ions2del,solvent_outside_box[i][4])

            if cation_resname in str(solvent_outside_box[i][3]) == True: #entry is cation
                
                cations = np.append(cations, solvent_outside_box[i])
                
            elif anion_resname in str(solvent_outside_box[i][3]) == True: #entry is anion
                
                anions = np.append(anions, solvent_outside_box[i])

            i = i + 1

        else: # entry is lipid atom

            lipids2del = np.append(lipids2del, solvent_outside_box[i][1])

            lipids = np.append(lipids, str(solvent_outside_box[i]))

            i = i + 1
            
        molecules2del = [water2del, ions2del, lipids2del]
        molecules = [cations, anions, lipids]    
            
    return molecules2del, molecules

In [72]:
molecules2del, molecules = delete_atoms_outside_box(solvent_outside_box, 'TIP3', 'NA', 'CL')

# Delete atoms outside box

<b> for now water and ions only are implemented </b>

In [74]:
water2del = molecules2del[0] # water residue numbers

In [75]:
len(water2del)

1886

In [76]:
ions2del = molecules2del[1] # ion residue numbers

In [77]:
if len(water2del) != 0: # only do the following if list has entries
    waters_outside = [] # indices of water atoms outside box
    
    for _water in water2del:
        
        water_outside_index = np.argwhere(residue_no == _water.astype('int')) # compare residue numbers of overlapping molecules and return their indices
        waters_outside = np.append(waters_outside, water_outside_index)

In [78]:
waters_outside

array([  25.,   26.,   27., ..., 6022., 6023., 6024.])

In [79]:
water2del

array([   4.,    5.,    6., ..., 2023., 2024., 2025.])

In [80]:
if len(ions2del) != 0: # only do the following if list has entries
    ions_outside = [] # indices of overlapping ions
    
    for _ion in ions2del:
        
        ions_outside_index = np.argwhere(residue_no == _ion.astype('int'))
        ions_outside = np.append(ions_outside, ions_outside_index)

## renumber and delete

<b> 3-atom water only!!! </b>

In [81]:
if len(water2del) != 0: # only do the following if list has entries
    
    # atoms before the first to be removed
    atom_data_new = atom_data[:waters_outside.astype('int')[0]] # ATOM
    atom_no_new = atom_no[:waters_outside.astype('int')[0]] # atom no.
    atom_type_new = atom_type[:waters_outside.astype('int')[0]] # atomtype
    residue_type_new = residue_type[:waters_outside.astype('int')[0]] # residue type
    residue_no_new = residue_no[:waters_outside.astype('int')[0]] # resid no.
    x_new = x[:waters_outside.astype('int')[0]] # x
    y_new = y[:waters_outside.astype('int')[0]] # y
    z_new = z[:waters_outside.astype('int')[0]] # z
    comments_new = comments[:waters_outside.astype('int')[0]] # rest

    atom_numbers = 0 # for atom numbers
    residue_numbers = 0 # for residue numbers
    
    while atom_numbers < len(waters_outside):
        
        if atom_numbers < len(waters_outside) - 5:
            
            atom_data_new = np.append(atom_data_new, atom_data[waters_outside.astype('int')[atom_numbers+2] + 1:waters_outside.astype('int')[atom_numbers+3]])
    
            atom_no_temp = atom_no[waters_outside.astype('int')[atom_numbers + 2 ] + 1:waters_outside.astype('int')[atom_numbers + 3]] # next block between atoms to remove   
            ablockshift = np.full(len(atom_no_temp), -3 - atom_numbers, dtype=int) # shift atom number by -3
            atom_no_temp = np.add(atom_no_temp, ablockshift) # mod atom no.s
            atom_no_new = np.append(atom_no_new, atom_no_temp) # append to atom no. list
    
            atom_type_new = np.append(atom_type_new, atom_type[waters_outside.astype('int')[atom_numbers + 2 ] + 1 :waters_outside.astype('int')[atom_numbers + 3 ]])
    
            residue_type_new = np.append(residue_type_new, residue_type[waters_outside.astype('int')[atom_numbers + 2 ] + 1 :waters_outside.astype('int')[atom_numbers + 3 ]])
    
            residue_no_temp = residue_no[waters_outside.astype('int')[atom_numbers + 2 ] + 1 :waters_outside.astype('int')[atom_numbers + 3 ]]
            rblockshift = np.full(len(atom_no_temp), -1 - residue_numbers, dtype=int) # shift residue number by -1
            residue_no_temp = np.add(residue_no_temp, rblockshift) # mod resid no.s
            residue_no_new = np.append(residue_no_new, residue_no_temp) # append to resid no. list
    
            x_new = np.append(x_new, x[waters_outside.astype('int')[atom_numbers + 2 ] + 1 :waters_outside.astype('int')[atom_numbers + 3 ]])
    
            y_new = np.append(y_new, y[waters_outside.astype('int')[atom_numbers + 2 ] + 1 :waters_outside.astype('int')[atom_numbers + 3 ]])
    
            z_new = np.append(z_new, z[waters_outside.astype('int')[atom_numbers + 2 ] + 1 :waters_outside.astype('int')[atom_numbers + 3 ]])
    
            comments_new = np.append(comments_new, comments[waters_outside.astype('int')[atom_numbers + 2 ] + 1 :waters_outside.astype('int')[atom_numbers + 3 ]])

    
            atom_numbers = atom_numbers + 3
            residue_numbers = residue_numbers + 1
        
        elif (atom_numbers >= len(waters_outside) - 5):
        
            atom_data_new = np.append(atom_data_new, atom_data[waters_outside.astype('int')[atom_numbers + 2 ]+1:])
    
            atom_no_temp = atom_no[waters_outside.astype('int')[ atom_numbers + 2 ]+1:] # last block after atoms to remove   
            ablockshift = np.full(len(atom_no_temp), -3 - atom_numbers, dtype=int) # shift atom number by -3
            atom_no_temp = np.add(atom_no_temp, ablockshift) # mod atom no.s
            atom_no_new = np.append(atom_no_new, atom_no_temp) # append to atom no. list
    
            atom_type_new = np.append(atom_type_new, atom_type[waters_outside.astype('int')[atom_numbers + 2 ]+1:])
    
            residue_type_new = np.append(residue_type_new, residue_type[waters_outside.astype('int')[atom_numbers + 2 ]+1:])
        
            residue_no_temp = residue_no[waters_outside.astype('int')[ atom_numbers + 2 ]+1:]
            rblockshift = np.full(len(residue_no_temp), -1 - residue_numbers, dtype=int) # shift residue number by -1
            residue_no_temp = np.add(residue_no_temp, rblockshift) # mod resid no.s
            residue_no_new = np.append(residue_no_new, residue_no_temp) # append to resid no. list
    
            x_new = np.append(x_new, x[waters_outside.astype('int')[atom_numbers + 2 ]+1:])
    
            y_new = np.append(y_new, y[waters_outside.astype('int')[atom_numbers + 2 ]+1:])
    
            z_new = np.append(z_new, z[waters_outside.astype('int')[atom_numbers + 2 ]+1:])
    
            comments_new = np.append(comments_new, comments[waters_outside.astype('int')[atom_numbers + 2 ]+1:])
    
            atom_numbers = atom_numbers + 3 

<b> 1-atom ions only!!! </b>

In [82]:
if len(ions2del) != 0 and len(water2del) != 0: # only do the following if the lists have entries
    
    # atoms before the first to be removed
    atom_data_new2 = atom_data_new[:ions_outside.astype('int')[0]] # ATOM
    atom_no_new2 = atom_no_new[:ions_outside.astype('int')[0]] # atom no.
    atom_type_new2 = atom_type_new[:ions_outside.astype('int')[0]] # atomtype
    residue_type_new2 = residue_type_new[:ions_outside.astype('int')[0]] # residue type
    residue_no_new2 = residue_no_new[:ions_outside.astype('int')[0]] # resid no.
    x_new2 = x_new[:ions_outside.astype('int')[0]] # x
    y_new2 = y_new[:ions_outside.astype('int')[0]] # y
    z_new2 = z_new[:ions_outside.astype('int')[0]] # z
    comments_new2 = comments_new[:ions_outside.astype('int')[0]] # rest

    atom_numbers = 0 # for atom numbers
    residue_numbers = 0 # for residue numbers
    
    while atom_numbers < len(ions_outside):
    
        if atom_numbers < len(ions_outside)-1:
    
            atom_data_new2 = np.append(atom_data_new2, atom_data_new[ions_outside.astype('int')[u+0]+1:ions_outside.astype('int')[atom_numbers + 1]])
    
            atom_no_temp = atom_no_new[ions_outside.astype('int')[atom_numbers + 0] + 1 : ions_outside.astype('int')[u+1]] # next block between atoms to remove   
            ablockshift2 = np.full(len(atom_no_temp), -1 - atom_numbers, dtype=int) # shift atom number by -3
            atom_no_temp = np.add(atom_no_temp, ablockshift2) # mod atom no.s
            atom_no_new2 = np.append(atom_no_new2, atom_no_temp) # append to atom no. list
    
            atom_type_new2 = np.append(atom_type_new2, atom_type_new[ions_outside.astype('int')[atom_numbers + 0] + 1 : ions_outside.astype('int')[atom_numbers + 1]])
    
            residue_type_new2 = np.append(residue_type_new2, residue_type_new[ions_outside.astype('int')[atom_numbers + 0] + 1 : ions_outside.astype('int')[atom_numbers + 1]])
        
            #data4442=np.append(data4442, data4444[ions_outside.astype('int')[atom_numbers + 0] + 1 : ions_outside.astype('int')[atom_numbers + 1]])
    
            residue_no_temp = residue_no_new[ions_outside.astype('int')[atom_numbers + 0] + 1 : ions_outside.astype('int')[atom_numbers + 1]]
            rblockshift2 = np.full(len(atom_no_temp), -1 - residue_numbers, dtype=int) # shift residue number by -1
            residue_no_temp = np.add(residue_no_temp, rblockshift2) # mod resid no.s
            residue_no_new2 = np.append(residue_no_new2, residue_no_temp) # append to resid no. list
    
            x_new2 = np.append(x_new2, x_new[ions_outside.astype('int')[atom_numbers + 0] + 1 :ions_outside.astype('int')[atom_numbers + 1]])
    
            y_new2 = np.append(y_new2, y_new[ions_outside.astype('int')[atom_numbers + 0] + 1 :ions_outside.astype('int')[atom_numbers + 1]])
    
            z_new2 = np.append(z_new2, z_new[ions_outside.astype('int')[atom_numbers + 0] + 1 :ions_outside.astype('int')[atom_numbers + 1]])
    
            comments_new2 = np.append(comments_new2, comments_new[ions_outside.astype('int')[atom_numbers + 0] + 1 :ions_outside.astype('int')[atom_numbers + 1]])
        
            
            atom_numbers = atom_numbers + 1
            residue_numbers = residue_numbers + 1
        
        elif atom_numbers == len(ions_outside)-1:
        
            atom_data_new2 = np.append(atom_data_new2, atom_data_new[ions_outside.astype('int')[atom_numbers + 0] + 1 :])
    
            atom_no_temp = atom_no_new[ions_outside.astype('int')[atom_numbers + 0] + 1 :] # next block between atoms to remove   
            ablockshift2 = np.full(len(atom_no_temp), -1 - atom_numbers, dtype=int) # shift atom number by -3
            atom_no_temp = np.add(atom_no_temp, ablockshift2) # mod atom no.s
            atom_no_new2 = np.append(atom_no_new2, atom_no_temp) # append to atom no. list
    
            atom_type_new2 = np.append(atom_type_new2, atom_type_new[ions_outside.astype('int')[atom_numbers + 0] + 1 :])
    
            residue_type_new2 = np.append(residue_type_new2, residue_type_new[ions_outside.astype('int')[atom_numbers + 0] + 1 :])
        
            #data4442=np.append(data4442, data4444[ions_outside.astype('int')[u+0]+1:])
    
            residue_no_temp = data53[ions_outside.astype('int')[atom_numbers + 0] + 1 :]
            rblockshift2 = np.full(len(atom_no_temp), -1 - residue_numbers, dtype=int) # shift residue number by -1
            residue_no_temp = np.add(residue_no_temp, rblockshift2) # mod resid no.s
            residue_no_new2 = np.append(residue_no_new2, residue_no_temp) # append to resid no. list
    
            x_new2=np.append(x_new2, x_new[ions_outside.astype('int')[atom_numbers + 0] + 1 :])
    
            y_new2=np.append(y_new2, y_new[ions_outside.astype('int')[atom_numbers + 0] + 1 :])
    
            z_new2=np.append(z_new2, z_new[ions_outside.astype('int')[atom_numbers + 0] + 1 :])
    
            comments_new2=np.append(comments_new2, comments_new[ions_outside.astype('int')[atom_numbers + 0] + 1 :])
        
    
            atom_numbers = atom_numbers + 1
        else:
            atom_numbers = atom_numbers + 1

# Save to File

In [83]:
if len(water2del) != 0: # only do the following if list has entries
    newdata = np.rec.fromarrays((atom_data_new, atom_no_new, atom_type_new, residue_type_new, residue_no_new, x_new, y_new, z_new, comments_new)) # overlapping atoms removed
    os.system('head -'+str(header_loc)+' '+input_path+' > smallbox_test.pdb') #pdb file header
    np.savetxt('smallbox_part.txt', newdata, fmt=('%4s', '%6i', '%4s', '%4s', '%4i', '%11.3f', '%7.3f', '%7.3f', '%20s')) #write box atoms to file
    os.system('cat smallbox_part.txt >> smallbox_test.pdb') #combine files, lipids
    os.system('tail -'+str(footer)+' '+input_path+' >> smallbox_test.pdb') #pdb file footer

In [84]:
if len(ions2del) != 0 and len(water2del) != 0 : # only do the following if lists have entries
    newdata2 = np.rec.fromarrays((atom_data_new2, atom_no_new2, atom_type_new2, residue_type_new2, residue_no_new2, x_new2, y_new2, z_new2, comments_new2)) # overlapping atoms removed
    os.system('head -'+str(header_loc)+' '+input_path+' > smallbox_test2.pdb') #pdb file header
    np.savetxt('smallbox_part.txt', newdata2, fmt=('%4s', '%6i', '%4s', '%4s', '%4i', '%11.3f', '%7.3f', '%7.3f', '%21s')) #write box atoms to file
    os.system('cat smallbox_part.txt >> smallbox_test2.pdb') #combine files
    os.system('tail -'+str(footer)+' '+input_path+' >> smallbox_test2.pdb') #pdb file footer

In [85]:
print('number of deleted water molecules: '+str(len(water2del)))
print('number of deleted cations: '+str(len(cations)))
print('number of deleted anions: '+str(len(anions)))
print('deleted lipids:')
print('lipid deletion not implemented')
#print(lipids)

number of deleted water molecules: 1886


NameError: name 'cations' is not defined

## put into small cell

In [86]:
from MDAnalysis.core.groups import *

In [87]:
u = mda.Universe('smallbox_test.pdb')

/home/ac127777/anaconda3/envs/mdanalysis-dev/lib/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:328: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


In [88]:
COG = AtomGroup.center_of_geometry(u.atoms)

In [89]:
COG

array([ 8.12456133, 22.05605717, 28.80960221])

In [90]:
u.atoms.positions

array([[ 7.853, 23.148, 28.251],
       [ 8.068, 21.493, 28.357],
       [ 9.008, 23.938, 28.829],
       ...,
       [15.333, 20.405, 22.041],
       [15.078, 21.081, 21.414],
       [15.403, 19.607, 21.518]], dtype=float32)

In [91]:
relative_coords = u.atoms.positions - COG

In [92]:
max_distance = []

In [93]:
for i,n in enumerate(np.transpose(relative_coords)):
    max_distance.append(np.max(relative_coords[:,i]))

In [94]:
boxedgelength = np.max(max_distance)*2

In [95]:
boxedgelength

16.316792594639416

In [96]:
box_cog = np.array([np.max(max_distance), np.max(max_distance), np.max(max_distance)])

In [97]:
box_cog

array([8.1583963, 8.1583963, 8.1583963])

In [98]:
move = COG - box_cog # vector which all atoms are moved along

In [99]:
move

array([-0.03383497, 13.89766088, 20.65120591])

In [100]:
positions_new = u.atoms.positions - move

In [101]:
u2 = u # copy to new universe

In [102]:
u2.atoms.positions = positions_new # set new positions

set new box to Universe.dimensions and the CRYST1 record when writing to pdb

In [103]:
u2.dimensions[:3] = np.array([boxedgelength,boxedgelength,boxedgelength])

In [104]:
u2.dimensions

array([16.316793, 16.316793, 16.316793, 90.      , 90.      , 90.      ],
      dtype=float32)

In [105]:
u2.atoms.write('smallbox.pdb')

/home/ac127777/anaconda3/envs/mdanalysis-dev/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'elements' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/ac127777/anaconda3/envs/mdanalysis-dev/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/ac127777/anaconda3/envs/mdanalysis-dev/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1176: UserWarning: Found missing chainIDs. Corresponding atoms will use value of 'X'
  warnings.warn("Found missing chainIDs."
